In [142]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [144]:
# ChromeDriver 자동 설치 및 실행
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 백그라운드 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Project Gutenberg 웹사이트 열기
driver.get("https://www.gutenberg.org/")
print("ChromeDriver 실행 성공!")

# 웹 요소가 상호 작용 가능한 상태가 될 때까지 최대 20초 동안 기다림
wait = WebDriverWait(driver, 20)

ChromeDriver 실행 성공!


In [146]:
# 검색창 찾기 (id="menu-book-search")
search_box = wait.until(EC.element_to_be_clickable((By.ID, "menu-book-search")))

# 사용자가 검색어 입력
search_term = input("검색할 단어를 입력하세요: ")

# 사용자가 입력한 검색어를 입력 후 검색 실행
search_box.send_keys(search_term)
search_box.send_keys(Keys.RETURN)
print(f"검색 완료: '{search_term}'")

# 검색 결과 페이지가 로드될 시간을 기다림
time.sleep(5)

검색할 단어를 입력하세요:  fiction


검색 완료: 'fiction'


In [116]:
# "Bookshelves" 버튼 찾기 및 클릭
bookshelves_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Bookshelves']")))
bookshelves_link.click()

In [117]:
# "Browsing: Fiction" 링크 찾기 및 클릭
browsing_fiction_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[@class='title' and text()='Browsing: Fiction']")))
browsing_fiction_link.click()

# 페이지가 로드될 시간을 기다림
time.sleep(5)

In [134]:
# 파일명에서 사용할 수 없는 문자 제거
def sanitize_filename(title):
    return re.sub(r'[\\/*?:"<>|\n]', '_', title).strip()

In [136]:
# 책 제목 리스트를 다시 가져오는 함수 (stale element reference 방지)
def reload_title_links():
    time.sleep(2)  # 페이지 로드 대기
    return [link for link in wait.until(EC.presence_of_all_elements_located((By.XPATH, "//*[@id='content']/div[2]/div/ul/li/a")))
            if 'Sort' not in link.text]

In [ ]:
try:
    # 책 제목 리스트 가져오기 (필터링하여 불필요한 링크 제외)
    title_links = reload_title_links()

    for index, link in enumerate(title_links):
        try:
            book_title = sanitize_filename(link.text)  # 파일명 정리
            print(f"{index+1}. {book_title} - 이동 중...")
            link.click()
            time.sleep(3)  # 페이지 로드 대기

            # 현재 페이지 URL 출력
            print(f"현재 페이지 URL: {driver.current_url}")

            # 'Read online (web)' 링크 클릭
            read_online_link = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Read online (web)")))
            read_online_link.click()
            time.sleep(3)  # 페이지 로드 대기

            # 본문 내용 크롤링
            book_content_elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//body//p")))
            book_text = "\n".join([para.text for para in book_content_elements])

            # 파일 저장
            file_path = f"{book_title}.txt"
            with open(file_path, "w", encoding="utf-8") as file:
                file.write(book_text)

            print(f"책 내용 저장 완료: {book_title}.txt 생성")

            # 📌 뒤로 가기 및 새로고침 (stale element reference 방지)
            driver.back()
            time.sleep(3)
            # title_links = reload_title_links()  # ✅ 리스트 새로 가져오기
            print('책 상세 내용으로 나가기')
            
            driver.back()
            time.sleep(3)
            # title_links = reload_title_links()  # ✅ 다시 리스트 새로 가져오기
            print('책 목록으로 나가기')
        
        except Exception as e:
            print(f"오류 발생 (책 제목: {book_title}): {e}")
            continue
except Exception as e:
    print(".")